In [1]:
import numpy as np # Successfully installed numpy-1.23.5
import pandas as pd # Successfully installed pandas-1.3.5
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Dataset
from torch_geometric.data import Data
# the temporal package is not available TODO: find what the issue is
# from torch_geometric_temporal.nn.recurrent import A3TGCN2 # torch-geometric-temporal 0.54.0 requires pandas<=1.3.5, but you have pandas 2.2.3 which is incompatible.
#from torch_geometric_temporal.signal import temporal_signal_split
# import 
# Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# from stellargraph import StellarGraph
# import custom functions from utils.py
# from utils import *
os.getcwd() # double check current working directory

'c:\\Users\\User1\\Documents\\GitHub\\CAN-Graph\\notebooks'

In [2]:
# .. goes up one level in directory
path = r'../datasets/Car-Hacking Dataset/Fuzzy_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/DoS_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/gear_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/RPM_dataset.csv'
df = pd.read_csv(path)
df.columns = ['Timestamp', 'CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8', 'label'] 

In [3]:
'''
Temporal graph dataset: a graph that changes over time
Graph type: Homogeneous graph
Nodes: CAN ID
Edges: The next CAN ID in the sequence
Node Features (optional): The data in the CAN ID

Steps:
1. identify node properties
2. Edges: How to connect nodes?
3. Extract labels

Convert to pytorch geometric format
One graph to another graph: 200 messages to 1 graph

There is also pytorch geometric temporal package

# build a baseline model RF, SVM, etc

# fraud datasets may have incorrect labels -> use label smoothing
Featureless nodes: use random numbers

make a GCN that uses labels only, and one that takes an average of data form its neighbors as well
Graph Attention Networks (GAT)

Idea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data
but it is in their best interest to have the best performing cybersecurity model.

https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.explain.algorithm.GNNExplainer.html
'''

"\nTemporal graph dataset: a graph that changes over time\nGraph type: Homogeneous graph\nNodes: CAN ID\nEdges: The next CAN ID in the sequence\nNode Features (optional): The data in the CAN ID\n\nSteps:\n1. identify node properties\n2. Edges: How to connect nodes?\n3. Extract labels\n\nConvert to pytorch geometric format\nOne graph to another graph: 200 messages to 1 graph\n\nThere is also pytorch geometric temporal package\n\n# build a baseline model RF, SVM, etc\n\n# fraud datasets may have incorrect labels -> use label smoothing\nFeatureless nodes: use random numbers\n\nmake a GCN that uses labels only, and one that takes an average of data form its neighbors as well\nGraph Attention Networks (GAT)\n\nIdea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data\nbut it is in their best interest to have the best performing cybersecur

In [ ]:
df['Node'] = df['CAN ID']
df['Edge'] = df['CAN ID'].shift(-1)

In [35]:
def hex_to_decimal(x):
    if x is None or x == 'None':
        return None
    try:
        return int(x, 16)
    except (ValueError, TypeError):
        return x


In [ ]:
# Drop the last row
df_dropped_last_row = df.drop(df.index[-1])

In [ ]:
# hex_to_decimal = lambda y: int(y, 16)
# Need to encode the strings to integers
specific_id_can_data_int = df.apply(lambda x: x.apply(hex_to_decimal))

In [ ]:
def create_graph(window_data, edge_index):
    x = torch.tensor(window_data, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return Data(x=x, edge_index=edge_index)

class TimeSeriesGraphDataset(Dataset):
    def __init__(self, time_series_data, window_size, stride):
        self.data = time_series_data
        self.window_size = window_size
        self.stride = stride
        self.graphs = self._create_graphs()

    def _create_graphs(self):
        graphs = []
        for i in range(0, len(self.data) - self.window_size + 1, self.stride):
            window = self.data[i:i+self.window_size]
            #print(window)
            graph = create_graph(window, self._get_edge_index(window[:, -1]))
            graphs.append(graph)
        return graphs

    def _get_edge_index(self, edge_data: np.ndarray) -> torch.Tensor:
        # Create edges based on the last column data
        edges = []
        for i, value in enumerate(edge_data):
            # Convert the edge value to an integer index
            target = int(value)
            if 0 <= target < len(edge_data):  # Ensure the target is within valid range
                edges.append([i, target])
        
        # If no valid edges, create a self-loop to avoid errors
        if not edges:
            edges = [[0, 0]]
        
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        return edge_index

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

In [ ]:
arr = df_dropped_last_row[['CAN ID', 'Node', 'Edge']].to_numpy()

In [ ]:
# # Extract edge indices
edge_index = torch.tensor(df[['Node', 'Edge']].values, dtype=torch.long)

# # Extract node features (assuming features are in columns 'feature1', 'feature2', ..., 'featureN')
node_features = torch.tensor(df[['Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8',]].values, dtype=torch.float)

y = torch.tensor(node_labels, dtype=torch.long)
# # Create a Data object
data = Data(x=node_features, edge_index=edge_index, y=y)

In [4]:
'''' From GitHub Copilot
Key Attributes of the Data Class:
x: Node feature matrix with shape [num_nodes, num_node_features]. Each row corresponds to the feature vector of a node.
edge_index: Graph connectivity in COO format with shape [2, num_edges]. Each column represents an edge between two nodes.
edge_attr (optional): Edge feature matrix with shape [num_edges, num_edge_features]. Each row corresponds to the feature vector of an edge.
y (optional): Graph-level or node-level labels.
pos (optional): Node position matrix with shape [num_nodes, num_dimensions]. Useful for visualization or spatial graphs.
batch (optional): Batch vector, which assigns each node to a specific graph in a mini-batch.


'''
# label logic - if T in window then label is 1, else 0. Start simple and build out from there
# x = [# nodes, # features], for my case [200, 10]
# edge_index = [2, # edges], for my case [2, 199]
# y = [# nodes, # labels], for my case [200, 1]

#dataset = TimeSeriesGraphDataset(arr, window_size=50, stride=1)

"' From GitHub Copilot\nKey Attributes of the Data Class:\nx: Node feature matrix with shape [num_nodes, num_node_features]. Each row corresponds to the feature vector of a node.\nedge_index: Graph connectivity in COO format with shape [2, num_edges]. Each column represents an edge between two nodes.\nedge_attr (optional): Edge feature matrix with shape [num_edges, num_edge_features]. Each row corresponds to the feature vector of an edge.\ny (optional): Graph-level or node-level labels.\npos (optional): Node position matrix with shape [num_nodes, num_dimensions]. Useful for visualization or spatial graphs.\nbatch (optional): Batch vector, which assigns each node to a specific graph in a mini-batch.\n\n\n"

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class AnomalyGNN(nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(AnomalyGNN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.classifier = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Aggregate node features
        return self.classifier(x).squeeze()
from torch_geometric.loader import DataLoader

dataset = TimeSeriesGraphDataset(time_series_data, window_size=50, stride=1)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = AnomalyGNN(num_node_features=50, hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()



model.eval()
with torch.no_grad():
    for batch in test_loader:
        predictions = torch.sigmoid(model(batch.x, batch.edge_index))
        anomalies = predictions > 0.5  # Binary classification threshold